In [135]:
#!/usr/bin/env python
import snowflake.connector
import time
import hashlib as hashlib
import pandas as pd
import numpy as np
import calendar
from snowflake.connector.pandas_tools import write_pandas
from snowflake.connector.pandas_tools import pd_writer
from hashlib import md5
###
# https://docs.snowflake.com/en/user-guide/python-connector-example.html
###
from datetime import date, datetime, timedelta
from dateutil.parser import parse as dateParse

In [136]:
# Create main connection context
con = snowflake.connector.connect(
    user='anuragvuppala',
    password='Snow@Flake44',
    account='vx24548.europe-west2.gcp'
)

In [137]:
c = con.cursor()
c.execute("SELECT current_version()").fetchone()
    

('6.2.1',)

In [138]:
c.execute("CREATE WAREHOUSE IF NOT EXISTS crimewh")
c.execute("USE WAREHOUSE crimewh")
c.execute('USE ROLE SYSADMIN')
c.execute('CREATE DATABASE IF NOT EXISTS CRIME_DB_2021')


In [139]:
c.execute('USE DATABASE CRIME_DB_2021')
c.execute('USE SCHEMA PUBLIC')

In [72]:
from glob import glob
filenames = glob('raw_datasets/*/2021*.csv',recursive=True) 
print(len(filenames))

520


In [140]:
from clean_data import clean_dataset
for file in filenames:
    # Read the data files individually 
    data = pd.read_csv(file)
    
    print('Reading file ==>' ,file)
    
    #Cleaning the data base for null values and replacing the with corrresponing valuse and droping unwanted columns
    
    data=clean_dataset(data)
    
    print('Cleaned file ==>',file)
    
    
    #Saving the cleaned file to to new directory
     
    p = 'processed_datasets/'+file[21:] 
    
    data.to_csv(p,header=None,index=False)
    
    print('Saving File ==>',file)
    
    
    

Reading file ==> raw_datasets/2021-09/2021-09-sussex-street.csv
Empty cells filled with respective values
unnecessary Columns removed
Cleaned file ==> raw_datasets/2021-09/2021-09-sussex-street.csv
Saving File ==> raw_datasets/2021-09/2021-09-sussex-street.csv
Reading file ==> raw_datasets/2021-09/2021-09-metropolitan-street.csv
Empty cells filled with respective values
unnecessary Columns removed
Cleaned file ==> raw_datasets/2021-09/2021-09-metropolitan-street.csv
Saving File ==> raw_datasets/2021-09/2021-09-metropolitan-street.csv
Reading file ==> raw_datasets/2021-09/2021-09-cleveland-street.csv
Empty cells filled with respective values
unnecessary Columns removed
Cleaned file ==> raw_datasets/2021-09/2021-09-cleveland-street.csv
Saving File ==> raw_datasets/2021-09/2021-09-cleveland-street.csv
Reading file ==> raw_datasets/2021-09/2021-09-norfolk-street.csv
Empty cells filled with respective values
unnecessary Columns removed
Cleaned file ==> raw_datasets/2021-09/2021-09-norfolk-s

In [141]:
c.execute('CREATE OR REPLACE TABLE "CRIME_DATA_2021" ("crime_id" STRING, "month" STRING, "reported_by" STRING, "fall_within" STRING, "longitude" FLOAT, "latitude" FLOAT, "location" STRING, "lsoa_code" STRING, "lsoa_name" STRING, "crime_type" STRING, "last_outcome" STRING)')

In [142]:
cleaned_filenames = glob('processed_datasets/2021*.csv') 
print(len(cleaned_filenames))

520


In [93]:
for file in filenames:
    # Read the data files individually 
    clean_data = pd.read_csv(file,header=None, skiprows=[1])
    write_pandas(con,clean_data,table_name="CRIME_DATA_2021")
    print('Table',clean_data, 'uploaded!')
    

ValueError: parquet must have string column names

In [144]:
c.execute("CREATE WAREHOUSE IF NOT EXISTS crimewh")
c.execute("USE WAREHOUSE crimewh")

In [151]:

c.execute("PUT file://processed_datasets/2021*.csv @%crime_data_2021")
c.execute("COPY INTO crime_data_2021")

In [147]:
c.execute('info loading_data').fetchall()

[('"The Snowflake online help is no longer being maintained. "For all product/feature information, please use https://docs.snowflake.com/en."\n',)]